## 0. 라이브러리

In [1]:
import os
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import shutil
import pickle
from tqdm import tqdm
from colorsys import rgb_to_hsv
from keras.preprocessing.image import load_img

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.optimize import linear_sum_assignment

from PIL import Image

from numpy import dot
from numpy.linalg import norm
import bisect

## 1. 이미지 파일명 불러오기

색 정보를 추출할 이미지의 파일명을 모두 불러온다.

##### get_file_names(img_path, ends, sample_ratio = 1)
- 이미지 경로 (img_path) 상에 있는 특정 확장자 파일명을 모두 읽어들인다.
 - sample_ratio 에 0~1값을 넣어 샘플링이 가능하다.
- 읽어들인 파일명을 리스트에 저장한 뒤 return 한다
- [ex] file_names = get_file_names('images/', 'jpg')

In [3]:
def get_file_names(img_path, ends, sample_ratio = 1):
    file_names = []

    with os.scandir(img_path) as files:
        for file in files:
            # .jpg .png 등 확장자명이 'g'로 끝나는 파일들 모두 읽기
            if file.name.endswith(ends):
                file_names.append(img_path + file.name)

    if sample_ratio < 1:
        file_names = random.sample(file_names, int(len(file_names) * sample_ratio))
        
    return file_names

## 2. 데이터 추출하기

이미지의 색 정보를 추출하는 방법에 대해 다양한 방법을 시도하였다. 기본적인 아이디어는 불러온 이미지로부터 각 픽셀의 색을 모두 구하는 것에서 시작된다. 가장 먼저 접근한 방식은 단순히 RGB 채널로 불러온 이미지의 픽셀들을 총 30개의 클러스터로 분류한 뒤, 작품 별 클러스터의 관계성으로부터 작품을 분류하고자 하였다. 그러나 RGB 채널의 값은 HSV 채널, YUV 채널에 비해 실제 우리가 눈으로 인식하는 색들을 표현하는 데 있어서 표현력이 좋지 않았으며 실제로 분류 결과도 타당하지 못했다. 따라서 우리는 여러 채널의 혼합 및 시행 착오를 겪을 필요가 있었고, 최종적으로 HSV채널과 YUV채널을 합친 데이터를 사용하기로 하였다. (2-3. HSV-YUV data)

### 2-1. HSV flatten data

##### extract_data_hsv_flatten(file_name, img_size = (360, 360), img_grid = 5, clr_grid = (12, 8, 8))
- cv2 라이브러리를 통해 RGB 채널로 읽어들인 이미지를 HSV로 변환한 뒤, 그리드에 맞게 팔레트 내의 색을 평면화 한다
 - 1. 먼저 이미지를 RGB 채널로 읽고, HSV채널로 변환한다. (각 채널별 range는 180,256,256 이다)
 - 2. 몇 분할로 나눌지를 파라미터로 넘겨받아 H, S, V를 각각 분해하고 평면화 한다.
 - 3. default 의 경우 H, S, V가 각각 12, 8, 8 개로 분해되어 그림 내의 픽셀들이 768 컬럼 중 한 군데로 분류된다.
- 평면화된 데이터를 return 한다 (default shape : (1, 768))
- [ex] flt_data = extract_data_hsv_flatten(file_names[target])

In [1]:
def extract_data_hsv_flatten(file_name, img_size = (360, 360), img_grid = 5, clr_grid = (12, 8, 8)):
    img = cv2.imread(file_name)
    img = cv2.resize(img, img_size)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    grid_count = np.zeros([img_grid * 2, clr_grid[0] * clr_grid[1] * clr_grid[2]])
    clr_data = []
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            newH = (int)((img_hsv[i][j][0] / 180) * clr_grid[0]) #range of H : 0 ~ 179
            newS = (int)((img_hsv[i][j][1] / 256) * clr_grid[1]) #range of S : 0 ~ 255
            newV = (int)((img_hsv[i][j][2] / 256) * clr_grid[2]) #range of V : 0 ~ 255
            label = newH * clr_grid[1] * clr_grid[2] + newS * clr_grid[2] + newV
            grid_count[(int)((i / img_size[0]) * img_grid)][label] += 1
            grid_count[img_grid + (int)((i / img_size[0]) * img_grid)][label] += 1
    for i, grid in enumerate(grid_count):
        grid_total = grid_count[i].sum()
        clr_data.append(grid_count[i] / grid_total) 
    return np.array(clr_data)

### 2-2. RGB-HSV data

##### extract_data_rgbhsv_before(file_name, img_size = (360, 360))
- keras 라이브러리를 통해 BGR 채널로 읽어들인 이미지를 rgb_to_hsv 함수를 통해 HSV 채널로 변환한다.
 - 이 때, 이미지는 1차원으로 reshape 해준다 (360x360x3→129600x3)
- 평면화된 RGB + HSV데이터를 return 한다 (default shape : (129600, 6))
- [ex] clr_data = extract_data_rgbhsv_before(file_names[target])

In [7]:
#tf.rgb_to_hsv version (구버전)
def extract_data_rgbhsv_before(file_name, img_size = (360, 360)):
    img = load_img(file_name, target_size = img_size)
    img = np.array(img)
    img = img.reshape(img_size[0] * img_size[1], 3)
    hsv_data = []
    for pixel in img:
        hsv_data.append(rgb_to_hsv(pixel[0] / 255, pixel[1] / 255, pixel[2] / 255))
    return np.concatenate((img / 255, hsv_data), axis = 1)

##### extract_data_rgbhsv(file_name, img_size = (360, 360))
- cv2 라이브러리를 통해 RGB 채널로 읽어들인 이미지를 HSV로 변환한다.
 - 이 때, 이미지는 1차원으로 reshape 해준다 (360x360x3→129600x3)
- 평면화된 RGB + HSV데이터를 return 한다 (default shape : (129600, 6))
- [ex] clr_data = extract_data_rgbhsv(file_names[target])

In [8]:
def extract_data_rgbhsv(file_name, img_size = (360, 360)):
    img = cv2.imread(file_name)
    img = cv2.resize(img, img_size) 
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).reshape(img_size[0] * img_size[1], 3)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).reshape(img_size[0] * img_size[1], 3)
    return np.concatenate((img_rgb, img_hsv), axis = 1)

### 2-3. HSV-YUV data

##### extract_data_hsvyuv(file_name, img_size = (360, 360))
- cv2 라이브러리를 통해 BGR 채널로 읽어들인 이미지를 HSV 및 YUV로 변환한다.
 - 이 때, 이미지는 1차원으로 reshape 해준다 (360x360x3→129600x3)
- 평면화된 HSV + YUV데이터를 return 한다 (default shape : (129600, 6))
- [ex] clr_data = extract_data_hsvyu(file_names[target])

In [9]:
def extract_data_hsvyuv(file_name, img_size = (360, 360)):
    img = cv2.imread(file_name)
    img = cv2.resize(img, img_size)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).reshape(img_size[0] * img_size[1], 3)
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV).reshape(img_size[0] * img_size[1], 3)
    return np.concatenate((img_hsv, img_yuv), axis = 1)

## 3. 클러스터링 데이터

이미지로 부터 추출해 낸 색 데이터를 총 몇개의 클러스터로 나눌 것인가에 대한 고민도 많이 이루어졌다. 클러스터의 개수를 유동적으로 조절하기에는 너무 많은 계산 시간을 필요로 하였으며, 클러스터 개수가 지나치게 많으면 추후 분류 작업에서 불필요한 색까지 모두 중요한 데이터로 인식되는 경향을 보였다. 반면 클러스터 개수가 지나치게 적으면 완전 다른 작품도 비슷한 작품으로 분류되는 경향성을 보였고 최종적으로 우리는 15~25개의 클러스터에서 가장 안정적인 결과를 구할 수 있었다.

### 3-1. Labels

##### get_clu_labels(data, cluster = 25, state = 2021)
- KMeans 군집화를 통해 전체 색 데이터에 대응하는 클러스터 레이블을 얻는다.
 - cluster 파라미터를 통해 레이블 개수를 조절할 수 있다.
- 클러스터 레이블을 return 한다 (default shape : (129600, 1))
- [ex] data_labels = get_clu_labels(clr_data)

In [10]:
def get_clu_labels(data, cluster = 25, state = 2021):
    km = KMeans(n_clusters = cluster, random_state = state)
    km.fit(data)
    return km.labels_

### 3-2. Visualization with each cluster

##### show_img_with_clusters(file_name, labels, img_size = (360, 360), grid_size = (5, 5), clu_color = (0, 255, 0))
- 각 군집 결과 별로 그림 상에 어느 부분에 해당하는지 시각화하여 보여준다.
 - grid_size : subplot 형태로 0번 클러스터부터 N번 클러스터까지 표현할 충분할 크기가 보장되어야 한다 (default : 5x5 = 25)
 - clu_color : 해당 클러스터를 어떤 색으로 표현할 지 RGB로 입력할 수 있다 (default : GREEN)
- [ex] show_img_with_clusters(file_names[target], data_labels)

In [8]:
def show_img_with_clusters(file_name, labels, img_size = (360, 360), grid_size = (5, 5), clu_color = (0, 255, 0)):
    plt.figure(figsize = (25, 25))
    img = load_img(file_name, target_size = img_size)
    img = np.array(img)
    
    for idx in range(grid_size[0] * grid_size[1]):
        img_now = img.copy()
        new_labels =  np.array(list(map(lambda x: x == idx, labels))).reshape(img_size)
        for i, row in enumerate(img):
            for j, col in enumerate(row):
                if new_labels[i][j]:
                    img_now[i][j] = clu_color
        plt.subplot(grid_size[1], grid_size[0] ,idx + 1);
        plt.imshow(img_now)
        plt.axis('off')

### 3-3. Visualization with grid

##### show_img_with_grid(file_name, labels, img_size = (360, 360), grid = (8, 8), cluster = 25, ratio = 0.1, clu_color = (0, 255, 0))
- 그림을 grid 개수에 맞게 분할한 뒤, 특정 클러스터가 해당 영역을 일정 비율만큼 차지하는지를 시각화하여 보여준다
 - grid : 그림을 얼만큼 분해할 지 정한다 (default : 8x8)
 - cluster : 레이블을 구할 때 입력했던 클러스터 개수를 입력해주어야 한다 (default : 25)
 - ratio : 넘지 못했을 때 손실시키기 위한 비율을 설정할 수 있다 (dafault : 0.1)
 - clu_color : 손실된 클러스터를 어떤 색으로 표현할 지 RGB로 입력할 수 있다 (default : GREEN)
- [ex] show_img_with_grid(file_names[target], data_labels, ratio = 0.3)

In [3]:
def show_img_with_grid(file_name, labels, img_size = (360, 360), grid = (8, 8), cluster = 25, ratio = 0.1, clu_color = (0, 255, 0)):
    plt.figure(figsize = (25, 25))
    img = load_img(file_name, target_size = img_size)
    img = np.array(img)
    new_labels =  labels.reshape(img_size)
    grid_count = np.zeros([grid[0] * grid[1], cluster])
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            grid_num = (int)(i / (img_size[0] / grid[0])) * grid[1] + (int)(j / (img_size[1] / grid[1]))
            grid_count[grid_num][new_labels[i][j]] += 1
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            grid_num = (int)(i / (img_size[0] / grid[0])) * grid[1] + (int)(j / (img_size[1] / grid[1]))
            if grid_count[grid_num][new_labels[i][j]] < (int)(img_size[0] * img_size[1] / grid[0] / grid[1] * ratio):
                img[i][j] = clu_color
    plt.imshow(img)
    plt.axis('off')   

## 4. 클러스터 데이터 추출

클러스터링을 통해 각 픽셀이 어떤 클러스터에 속하는지를 알 수 있게 되었는데, 우리는 이를 통해 각 클러스터가 실제 작품의 어느 부분에 존재하는지를 관찰할 수 있었다. 그 중에서는 한 군데에 밀집된 클러스터가 존재하는 반면 동시에 작품의 곳곳에 산개한 클러스터도 존재하였으며, 작품 비율의 대부분을 차지하는 클러스터도 존재하였다. 이러한 정보를 반영하기 위해 우리는 각 클러스터 별 색의 평균값과 클러스터의 중심 좌표, 그리고 모든 픽셀들이 중심으로부터 얼마나 떨어져있는지를 평균 거리 및 거리의 표준 편차를 통해 확인하고 최종적으로 픽셀 개수를 통해 클러스터가 작품에 차지하는 비율을 구하였다.

### 4.1. 클러스터 RGB평균, 중심 좌표, 평균 거리, 평균 표준편차, 비율

##### grouping_clusters(file_name, labels, img_size = (360, 360), cluster = 25)
- 각 클러스터 레이블을 기준으로 다양한 정보를 추출한다.
 - 1. 레이블의 RGB 평균 값 (0, 1, 2 columns)
 - 2. 이미지 상에서 중심 좌표 (3, 4 columns)
 - 3. 중심 좌표로부터의 픽셀 거리들의 평균 (5 column)
 - 4. 중심 좌표로부터의 픽셀 거리들의 표준편차 (6 column)
 - 5. 해당 클러스터가 작품에서 차지하는 비율 (7 column)
- 추출한 정보를 취합하여 return 한다. (default shape : (25, 7))
- [ex] clu_data = grouping_clusters(file_names[target], data_labels)

In [6]:
def grouping_clusters(file_name, labels, img_size = (360, 360), cluster = 25):
    img = cv2.imread(file_name)
    img = cv2.resize(img, img_size)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).reshape(img_size[0] * img_size[1], 3)
    
    rgbs = np.array([img_rgb[labels == i].mean(axis = 0) for i in range(cluster)])
    
    center_info = np.zeros([cluster, 3])
    new_labels = labels.reshape(img_size[0], img_size[1])
    for i in range(img_size[0]):
        for j in range(img_size[1]):
            tar = new_labels[i][j]
            center_info[tar] += [i, j, 1]    
    centers = np.array([[x / z,y / z] for [x,y,z] in center_info])
    ratios = np.array([[z / (img_size[0] * img_size[1])] for [x,y,z] in center_info])
    
    dist_info = [[] for i in range(cluster)]
    for i in range(img_size[0]):
        for j in range(img_size[1]):
            tar = new_labels[i][j]
            dist_info[tar].append(norm(centers[tar] - [i, j]))
    dists = np.array([[np.mean(data), np.std(data)] for data in dist_info])

    return np.concatenate((rgbs, centers, dists, ratios), axis = 1)

### 4-2. Visualization with cluster quality

##### show_img_with_cluster_quality(file_name, labels, clu_data, img_size = (360, 360))
- 그림에 존재하는 모든 색을 클러스터의 평균 색으로 치환한다. 즉 그림을 클러스터 개수만큼의 색으로 재표현한다.
 - 4-1 에서 구한 RGB 평균 값을 활용한다
- [ex] show_img_with_cluster_quality(file_names[target], data_labels, clu_data)

In [ ]:
def show_img_with_cluster_quality(file_name, labels, clu_data, img_size = (360, 360)):
    plt.figure(figsize = (25, 25))
    img = load_img(file_name, target_size = img_size)
    img = np.array(img)
    
    new_labels =  labels.reshape(img_size)
    for i, row in enumerate(img):
        for j, col in enumerate(row):
            img[i][j] = clu_data[new_labels[i][j]][0 : 3]
    plt.imshow(img)
    plt.axis('off')

### 4-3. Visualization with color ratio

##### show_clu_by_ratio(clu_data)
- 각 클러스터 레이블이 그림 내에서 어떤 색을 대표하고, 그림 내의 비율이 얼마인지를 시각화한다.
 - barh plot 형태로 시각화한다.
 - 4-1 에서 구한 RGB 평균 값, 비율 값을 활용한다
- [ex] show_clu_by_ratio(clu_data)

In [10]:
def show_clu_by_ratio(clu_data):
    plt.figure(figsize = (8, 8))
    clu_data_sorted = clu_data[np.argsort(clu_data[:, 7])]
    plt.barh(range(len(clu_data)), clu_data_sorted[:, 7], color = clu_data[:, 0:3].astype(int)/256)
    plt.show()

### 4-4. Visualization with color position

##### show_clu_by_pos(clu_data)
- 각 클러스터 레이블의 중심이 그림에서 어디에 존재하는지를 시각화한다.
 - scatter plot 형태로 시각화한다.
 - 4-1 에서 구한 RGB 평균 값, 비율 값을 활용한다
 - scatter 의 크기는 비율 값에 비례하도록 표현하였다.
- [ex] show_clu_by_pos(clu_data)

In [11]:
def show_clu_by_pos(clu_data):
    plt.figure(figsize = (8, 8))
    plt.xlim([0, 360])
    plt.ylim([0, 360])
    plt.gca().invert_yaxis()
    plt.scatter(x = clu_data[:, 4], y = clu_data[:, 3], s= clu_data[:,7] * 10000, c = clu_data[:, 0:3].astype(int)/256)
    for i in range(len(clu_data)):
        plt.text(x = clu_data[i, 4] - 1.2, y = clu_data[i, 3] + 0.4, s = str(i), size = 'small')
    plt.show()

## 5. 데이터 저장

클러스터링은 n=20 기준으로 약 10시간이 걸릴 정도로 오랜 작업을 요구했다. 따라서 이 후 분류 작업을 원활하게 수행하기 위해 데이터를 추출한 후 저장하는 과정을 포함해야 했다.

##### save_clu_data(save_path, file_name, data)
- 그림으로 부터 추출한 클러스터 정보를 저장한다
 - 저장 경로에 해당하는 디렉토리가 없으면 만들어준다
 - 이미지 파일 명을 가져와 확장자만 npy로 변경하여 저장한다
- [ex] save_clu_data('my_save/', file_names[target], clu_data)

In [ ]:
def save_clu_data(save_path, file_name, data):
    try:
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    except OSError:
        print ('Error: Creating directory. ' +  save_path)
    try:
        save_name = save_path + file_name.rsplit('.')[0].rsplit('/')[-1] + ".npy"
        np.save(save_name, data)
    except:
        print ('Error: Creating Data. ' + save_name)

## 1~5. Extract Color

##### extract_color_v1(img_path, save_path, img_size = (360, 360), clu = 25)
- 이미지로부터 클러스터 정보를 읽어오는 과정을 취합하였다.
 - 저장 경로에 이미 해당 데이터가 존재하면 과정을 생략하도록 설계하였다.
- [ex] extract_color_v1('images/', 'my_save/', clu = 20)

In [9]:
def extract_color_v1(img_path, save_path, img_size = (360, 360), clu = 25):
    file_names = get_file_names(img_path, 'jpg', 1)
    for target_name in tqdm(file_names):
        save_name = save_path + target_name.rsplit('.')[0].rsplit('/')[-1] + ".npy"
        if not os.path.isfile(save_name):
            hsvyuv_data = extract_data_hsvyuv(target_name, img_size = img_size)
            data_labels = get_clu_labels(hsvyuv_data, cluster = clu)
            clu_data = grouping_clusters(target_name, data_labels, cluster = clu)
            save_clu_data(save_path, target_name, clu_data)

## 6. 클러스터 데이터 로드

이미지 별로 기존에 작업했던 클러스터 데이터를 모두 로드한다. 이 때 이미지의 개수와 클러스터 행렬의 행 개수가 일치하는지 반드시 확인할 필요가 있다.

##### load_clu_data(data_path, sample_ratio = 1)
- 데이터 경로 (data_path) 상에 있는 .npy 확장자 파일명을 모두 로드한다.
 - sample_ratio 에 0~1값을 넣어 샘플링이 가능하다.
- 읽어들인 파일명을 ndarray 형식으로 return 한다 (default shape : (5411, 25, 7)) *이미지 개수 = 5411
- [ex] file_names = get_file_names('images/', 'jpg')

In [ ]:
def load_clu_data(data_path, sample_ratio = 1):
    clu_data = []

    with os.scandir(data_path) as files:
        for file in files:
            if file.name.endswith('npy'):
                clu_data.append(np.load(file))
    if sample_ratio < 1:
        clu_data = random.sample(clu_data, int(len(clu_data) * sample_ratio))
        
    return np.array(clu_data)

## 7. 데이터 평면화 작업

각 이미지마다 클러스터의 개수 및 각 클러스터의 특징값을 지니고 있었기 때문에 이미지는 2차원의 데이터로 표현되었다. 여기에 이미지의 개수를 포함하면 데이터는 3차원이 되었고 이는 머신 러닝 데이터로 적합하지 않았다. 따라서 우리는 데이터를 1차원으로 평면화하는 작업을 수행할 필요가 있었다. 평면화 작업은 각 클러스터가 가진 RGB 값과 클러스터의 비율을 기반으로 수행되었다. RGB값이 0~1로 표현되었을 때, 이를 적절하게 k개로 분할해주면 k * k * k 개의 인덱스로 표현이 가능했다. k가 너무 크면 sparce space 가 너무 많이 생기고, k가 너무 적으면 다른 색이어도 동일한 인덱스에 할당될 가능성이 있었기 때문에 주의가 필요했다. 우리는 30개의 클러스터를 5x5x5 = 125개의 인덱스에 할당하는 것이 가장 적합하다고 판단하였고 k=4 또는 k=8일때보다 훨씬 괜찮은 성능을 보이는 것을 관찰할 수 있었다.

##### flatten_clusters_rgb(clu_data, div = 5)
- 클러스터의 RGB 평균값을 기준으로 그리드에 맞게 색을 평면화 한다
 - div : 몇 분할로 나눌지를 파라미터로 넘겨받는다. (default : 5, 5x5x5 →125 columns)
 - 시각에 민감한 색이 G, R, B 순임으로 차수를 이와 같이 정해 민감한 색의 컬럼을 최대한 떨어트린다.
- 평면화된 데이터를 return 한다 (default shape : (5411, 125))
- [ex] flt_data = flatten_clusters_rgb(clu_data)

In [1]:
def flatten_clusters_rgb(clu_data, div = 5):
    flt_data = []
    for feature in clu_data: 
        R = ((feature[:,0] / 256) * div).astype(int)
        G = ((feature[:,1] / 256) * div).astype(int)
        B = ((feature[:,2] / 256) * div).astype(int)
        tar = G * div * div + R * div + B
        new_row = np.zeros(div ** 3)
        for num, val in zip(tar, feature[:, -1]):
            new_row[num] += val
        flt_data.append(new_row)
    return np.array(flt_data)

## 8. 차원 축소 (PCA)

데이터의 컬럼 수가 여전히 많으므로 차원 축소 기법을 수행하였다.

##### reduction_feature_pca(flt_data, threshold = 0.99)
- PCA 기법을 통해 데이터의 컬럼(차원)을 축소한다.
 - threshold : 정보 유지율을 파라미터로 전달받는다 (default : 0.99)
- 변환된 데이터를 return 한다 (default shape : (5411, d))
- [ex] pca_data = reduction_feature_pca(flt_data)

In [ ]:
def reduction_feature_pca(flt_data, threshold = 0.99):
    pca = PCA(n_components = len(flt_data[0]), random_state = 2021)
    pca.fit_transform(flt_data)
    
    ratio = np.cumsum(pca.explained_variance_ratio_)
    d = np.argmax(ratio >= threshold) + 1
    
    pca = PCA(n_components = d, random_state = 2021)
    return pca.fit_transform(flt_data)

## 9. 작품 클러스터링 및 그룹화

작품마다 생성된 색상 행렬을 기반으로 작품의 클러스터링을 수행하였다. 처음에 우리는 작품의 색 구성이 단조로운 작품들이 구성이 풍부한 작품보다 계층이 높을 필요가 있다고 판단하여 계층적 클러스터링 (hierarchical clustering) 및 밀도 기반 클러스터링을 수행하였다. 그러나 작품이 내재한 색상의 다양성은 이러한 클러스터로 모두 표현할 수 없었는데, 두 가지 클러스터 기법의 경우 특정 클러스터에 매우 적은 작품 수를 할당하는 문제가 발생하였다. 따라서 우리는 이를 단순 유클리드 거리 기반으로 한 K-means 알고리즘을 통해 해소하고자 하였으며 결과는 기존의 클러스터링 기법에 비해 준수한 분포를 이루었다.

### 9-1. Labels (3-1 과 동일)

##### get_clu_labels(data, cluster = 25, state = 2021)
- KMeans 군집화를 통해 각 작품 별 클러스터 레이블을 얻는다.
 - cluster 파라미터를 통해 레이블 개수를 조절할 수 있다. 작품 수가 5411개 이므로 clu = 50 이상의 환경에서 분류를 수행할 필요가 있다.
- 클러스터 레이블을 return 한다 (default shape : (129600, 1))
- [ex] data_labels = get_clu_labels(clr_data)

### 9-2. Visualization with groups

##### show_img_by_group(img_path, labels, group_num, grid = (7, 3), img_size = (360, 360))
- 작품 별 레이블을 기반으로 동일한 레이블을 가진 작품들을 그룹으로 보여준다
 - grid = 작품을 subplot 형식으로 보여주며 개수를 조절할 수 있다 (default = (7, 3) → 21개)
- [ex] show_img_by_group(file_names, data_labels, 0)

In [ ]:
def show_img_by_group(img_path, labels, group_num, grid = (7, 3), img_size = (360, 360)):
    plt.figure(figsize = (grid[0] * 3, grid[1] * 3))
    group = []
    for path, label in zip(img_path, labels):
        if label == group_num:
            group.append(path)
    
    print(f"Cluster size : {len(group)}")
    for idx in range(min(grid[1] * grid[0], len(group))):
        
        plt.subplot(grid[1], grid[0] ,idx+1);
        img = load_img(group[idx], target_size = img_size)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

## 10. 작품 저장

분류 결과를 직접 확인하기 위해 폴더 별로 동일한 클러스터를 가진 이미지를 저장한 후 관찰하였다.

##### save_groups(save_path, img_path, labels)
- 작품 별 레이블을 기반으로 작품들을 분류하여 저장한다.
 - shutil 라이브러리의 copyfile 을 통해 이미지를 복사해오는 방식으로 저장하였다.

In [ ]:
def save_groups(save_path, img_path, labels):
    for img, label in zip(img_path, labels):
        result_path = save_path + str(label) + "/"
        try:
            if not os.path.exists(result_path):
                os.makedirs(result_path)
        except OSError:
            print ('Error: Creating directory. ' +  result_path)   
        shutil.copyfile(img, result_path + img.rsplit('/')[1])

## 6~10. Clustering Images

##### clustering_images_v1(data_path, img_path, result_path, clu)
- 클러스터 정보로부터 이미지를 분류하는 과정을 취합하였다.
 - data_path : 불러올 작품 클러스터 정보가 존재해야 한다.
 - img_path : 불러올 작품이 존재하며 data_path 상에 있는 데이터와 크기 및 이름 정보가 일치해야 한다
 - clu : 분류 개수를 정할 수 있다. (default = 50)
- [ex] clustering_images_v1('my_save/', 'images/', 'my_results/')

In [4]:
def clustering_images_v1(data_path, img_path, result_path, clu = 50):
    clu_data = load_clu_data(data_path)
    flt_data = flatten_clusters_rgb(clu_data)
    pca_data = reduction_feature_pca(flt_data)
    img_names = get_file_names(img_path, 'jpg')
    data_labels = get_clu_labels(pca_data, cluster = clu)
    save_groups(result_path, img_names, data_labels)

## 11. 특징맵 생성 작업

추천 시스템을 구현하는 것은 작품을 분류하는 것과는 다른 특징을 필요로 하였다. Content-Based Filtering 을 수행하기 위해서는 작품 간의 관계를 파악하는 특징 값이 필요로 하였고, 우리는 두 작품의 특징을 하나의 값으로 표현하여 작품 수가 행과 열이 되는 정방행렬을 만드는 것을 새로운 목표로 하였다. 가장 먼저 접근한 방식은 코사인 유사도로, 평면화된 1차원 색상 행렬 두 개의 코사인 유사도를 저장하는 방식으로 정방행렬을 구성하였다. (11-2. Feature map with Cosine Similarity) 두번째는 아직 평면화되지 않은 색상 행렬에서 두 클러스터 집단의 1:1 매칭을 수행하는 방식으로 최소 비용을 저장하는 방식으로 정방행렬을 구성하였다. 이는 단순히 유량 문제 (Networkflow Problem) 으로 치환할 수 있었으나 시간 복잡도가 O(N^2 * C^5) 에 다다르는 결과를 초래하였다. 조사 결과 우리는 할당 문제 (Assignment Problem) 를 해결하는 헝가리안 알고리즘(Hungarian Algorithm) 및 이를 지원하는 라이브러리를 찾을 수 있었다. (11-4-4. Using linear_sum_assignment Function)

### 11-1. Cosine Simillarity

##### cos_sim(A, B)
- 코사인 유사도 결과를 return 한다.
 - A, B : 차원이 동일한 1차 행렬
- [ex] fmap[i][j] = cos_sim(data[i], data[j])

In [2]:
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

### 11-2. Feature map with Cosine Similarity

##### feature_map_cossim(data)
- 2차원으로 구성된 데이터에서 두 행 간의 코사인 유사도를 구한다 (*이미지 개수 x 이미지 특징 벡터)
- 행의 개수와 일치한 정방행렬을 return 한다. (default : N * N, N = 이미지 개수)
- [ex] fmap = feature_map_cossim(data)

In [3]:
def feature_map_cossim(data):
    N = len(data)
    fmap = np.zeros([N, N])
    for i in tqdm(range(N)):
        for j in range(N):
            fmap[i][j] = cos_sim(data[i], data[j])
    return fmap

### 11-3. Feature map scailing with Square Function

##### square_filter(val, m, M, reverse = False)
- 최솟값이 m, 최댓값이 M 인 1차 행렬을 0~1 값으로 스케일링 했을 때의 val값을 알려준다.
 - val : 실제 구하고자 하는 input 값이다.
 - reverse : True이면 최솟값이 1, 최댓값이 0으로 스케일링이 이루어진다.
- 사용되는 함수는 계수가 1인 x^2 함수이다.
- 스케일링된 값을 return 한다.
- [ex] new_val[j] = sqaure_filter(data[i][j], data[i].min(), data[i].max())
 

In [ ]:
def square_filter(val, m, M, reverse = False):
    if reverse:
        return ((val - M) ** 2) / ((m - M) ** 2)
    else:
        return ((val - m) ** 2) / ((m - M) ** 2)

##### feature_map_dist_scaled(data, sigma = 0.5)
- square_filter 를 기반으로 거리값을 구한 행렬을 0~1값으로 scailing 한다.
 - data[i][i] (동일한 데이터) 의 경우 거리 값이 무조건 0이 된다.
 - 이를 두번째 최솟값에서 표준편차x시그마 를 빼준 값으로 치환한다.
 - 최솟값이 0, 최댓값이 1인 reversed square filter 를 적용한다.
- 스케일링된 행렬을 return 한다.
- [ex] fmap = feature_map_dist_scaled(dist_data)
  

In [ ]:
def feature_map_dist_scaled(data, sigma = 0.5):
    N = len(data)
    for i in range(N):
        for j in range(i, N):
            data[j][i] = data[i][j]
    for i in range(N):
        data[i][i] = np.unique(data[i])[1] - sigma * np.std(data[i])
    return np.array([square_filter(nn, nn.min(), nn.max(), reverse = True) for nn in data])

### 11-4. Feature map with Cluster Matching based on Minimal Cost Matching

#### 11-4-1. Init

##### network_flow_init(clu = 25)
- 유량 문제를 해결하기 위한 초기화 작업을 수행하였다.
 - 용량, 유량, 비용 행렬을 초기화 한다
 - 클러스터간 fully-connected layer를 생성한다
- [ex] network_flow_init()

In [5]:
def network_flow_init(clu = 25):
    global capacity, flow, cost, adj, INF, source, sink
    v = 2 * clu + 2
    capacity = [[0]*v for _ in range(v)] # 용량
    flow = [[0]*v for _ in range(v)] # 유량
    cost = [[0]*v for _ in range(v)] # 비용
    adj = [[] for _ in range(v)] # 인접 그래프
    INF = 9876543210
    source = 2 * clu
    sink = 2 * clu + 1

    for i in range(clu):
        adj[source].append(i)
        adj[i].append(source)
        capacity[source][i] = 1
    for i in range(clu, 2 * clu):
        adj[sink].append(i)
        adj[i].append(sink)
        capacity[i][sink] = 1
    for i in range(clu):
        for j in range(clu, 2 * clu):
            adj[i].append(j)
            adj[j].append(i)
            capacity[i][j] = 1

#### 11-4-2. Cost Function

##### get_cost(now, tar)
- 두 클러스터 간의 비용을 얼마로 할지를 정의하였다.
 - 기본적으로 유클리드 거리를 비용으로 상정하였으며, 작품이 가진 특징의 종류 별로 다른 가중치를 적용하여 비용 함수를 정의하였다.
 - w1 = 8 : RGB 채널이 작품의 인상을 가장 크게 결정짓기 때문에 가장 높은 비율을 할당할 필요가 있었다.
 - w2 = 3 : 우리는 작품 상에서 클러스터의 위치 또한 작품의 인상에 큰 요인을 준다는 것을 분류 작업을 통해 느낄 수 있었고, 이에 대한 가중치를 적용하였다.
 - w3 = 2 : 클러스터가 얼마나 산개해있는지를 나타내는 평균, 분산 값의 차이는 사실 상 매우 난해한 값으로 작용하였다. 이에 대해 적절한 가중치를 부여하였다.
 - w4 = 800 : 각 클러스터가 차지하는 비율은 n=25 기준 평균 4%에 불과하였고, 이들의 차이는 가장 많은 클러스터를 제외하면 0.01%p 정도에 불과했다. 이를 거리로 표현하면 매우 낮은 수치에 해당하였고 이를 어느정도 높일 필요가 있었다.
- [ex] cost[i][j] = get_cost(data[i], data[j])

In [6]:
def get_cost(now, tar):
    diff_color = 8 * norm(now[0:3] - tar[0:3]) # RGB 차이
    diff_pos = 3 * norm(now[3:5] - tar[3:5]) # 중심 좌표 차이
    diff_dist = 2 * norm(now[5:7] - tar[5:7]) # 평균, 분산 차이
    diff_ratio = 800 * norm(now[-1] - tar[-1]) # 비율 차이
    return diff_color + diff_pos + diff_dist + diff_ratio

#### 11-4-3. MCMF Algorithm

#### network_flow(now, tar, clu = 25)
- 유량 문제를 해결하기 위한 알고리즘으로, 두 작품에 내재한 클러스터 간 1:1 매칭을 모두 수행하였을 때의 최소 비용을 계산한다.
 - 정점이 곧 클러스터 수이며, 모든 클러스터가 반대쪽 클러스터와 매칭될 가능성이 있으므로 간선의 개수는 (클러스터 수 ^ 2)이다.
 - 해당 유량 알고리즘의 시간 복잡도는 O(VE^2) 이므로 클러스터 개수를 C라고 하였을 때 최종 시간복잡도는 O(C^5) 이다.
- 특징 맵을 구현하기 위해서는 N * N 번의 작업이 필요하므로 특징 맵 구현을 위한 시간복잡도는 O(N^2C^5)이다.
- 두 작품의 클러스터 간 1:1 매칭을 수행하였을 때의 최소 비용을 return 한다.
- [ex] fmap[x][y] = network_flow(x, y)

In [ ]:
def network_flow(now, tar, clu = 25):
    global capacity, flow, cost, adj, INF, source, sink
    v = 2 * clu + 2
    flow = [[0]*v for _ in range(v)] # 유량
    cost = [[0]*v for _ in range(v)] # 비용
    for i in range(clu):
        for j in range(clu, 2 * clu):
            cost[i][j] = get_cost(now[i], tar[j - clu])
            cost[j][i] = -cost[i][j]
        
    answer = [0, 0] # 최소 비용, 최대 유량
    while True:
        path, dist = [-1]*v, [INF]*v
        inQueue, queue = [0]*v, [source] # 다음에 방문할 정점들
        dist[source], inQueue[source] = 0, 1
        while queue:
            present = queue[0] # 현재 정점
            del queue[0]
            inQueue[present] = False
            for _next in adj[present]:
                # 최소 비용이고, 최대 유량일 경우
                if dist[_next] > dist[present] + cost[present][_next] and capacity[present][_next] - flow[present][_next] > 0:
                    dist[_next], path[_next] = dist[present] + cost[present][_next], present
                    if not inQueue[_next]:
                        queue.append(_next)
                        inQueue[_next] = 1
        if path[sink] == -1: # 가능한 모든 경로를 찾았을 경우
            break
        # 현재 경로에서의 최소 유량 찾음
        flowRate = INF
        present = sink
        while present != source:
            previous = path[present]
            flowRate = min(flowRate, capacity[previous][present] - flow[previous][present])
            present = path[present]
        # 유량 흘림
        present = sink
        while present != source:
            previous = path[present]
            answer[0] += flowRate*cost[previous][present] # 총 비용이 각 간선 비용만큼 증가
            flow[previous][present] += flowRate
            flow[present][previous] -= flowRate # 음의 유량
            present = path[present]
        answer[1] += flowRate
    return answer

#### 11-4-4. Using linear_sum_assignment Function

##### feature_row_matching(data, now, clu = 25)
- scipy 라이브러리를 통해 할당 문제를 해결한다.
 - fully-connected 이고, 각 정점을 1:1로 매칭하는 이분 매칭 문제는 할당 문제(Assignment Problem)으로 치환이 가능하다.
 - 이는 시간복잡도가 O(N^3) 인 헝가리안 알고리즘으로 해결 가능하다.
 - 특징 맵을 구현하기 위해서는 N * N 번의 작업이 필요하므로 특징 맵 구현을 위한 시간복잡도는 O(N^2C^3)이다.
 - 기존 유량 문제에 비해 약 10배 빠른 성능으로 계산할 수 있었다.
- 실제 feature map 을 생성할 때는 fmap[i][j] = fmap[j][i] 임을 이용하여 삼각행렬만을 계산하였다.
- 한 작품이 N개의 작품에 대응하는 최소 비용 행렬 (1:N) 을 return한다.

In [ ]:
def feature_row_matching(data, now, clu = 25):
    N = len(data)
    clu = len(data[0])
    fmap = np.zeros(N)

    for i in tqdm(range(N)):
        cost_table = np.zeros([clu, clu])
        for x, clu_now in enumerate(data[i]):
            for y, clu_tar in enumerate(data[now]):
                cost_table[x][y] = get_cost(clu_now, clu_tar)
        row_ind, col_ind = linear_sum_assignment(cost_table)
        answer = 0
        for p, q in zip(row_ind, col_ind):
            answer += cost_table[p][q]
        fmap[i] = answer
    return fmap

## 12. 선호도 기반 이미지 인덱스 샘플링

### 12-1. 단순 선호도 행렬 기반

In [ ]:
def get_random_idx(scores, peak, length = 9):
    result = []
    ptg = peak ** scores
    ptg = np.cumsum(ptg)
    while len(result) != length:
        pt = random.random() * ptg[-1]
        idx = bisect.bisect_right(ptg, pt)
        if scores[idx] != 1 and idx not in result:
            result.append(idx)
    return result

### 12-2. 교차 선호도 행렬 기반

In [ ]:
def get_random_idx_cross(scores, rscores, peak, length = 9):
    result = []
    ptg = peak ** (scores - rscores)
    ptg = np.cumsum(ptg)
    while len(result) != length:
        pt = random.random() * ptg[-1]
        idx = bisect.bisect_right(ptg, pt)
        if scores[idx] != 1 and idx not in result:
            result.append(idx)
    return result

### 12-3. Visualization of sampled index

In [ ]:
def show_image_sampled(file_names, idx_list, plotsize = (3, 3), figsize = (12,12)):
    plt.figure(figsize = figsize);
    for i, idx in enumerate(idx_list):
        plt.subplot(plotsize[0], plotsize[1], i + 1)
        img = load_img(file_names[idx])
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
    plt.pause(0.3)    

## 13. 입력 결과에 따른 사용자 선호도 갱신

### 13-1. 단순 선호도 및 최댓값 기반

In [ ]:
def get_new_score_linearmax(data, scores, idx_list, val_list):
    for idx, val in zip(idx_list, val_list):
        if val > 0:
            scores = np.array([max(x, y) for x, y in zip(scores, data[idx])])
    return scores

### 13-2. 교차 선호도 및 최댓값 기반

In [ ]:
def get_new_score_crossmax(data, scores, rscores, idx_list, val_list):
    for idx, val in zip(idx_list, val_list):
        if val > 0:
            scores = np.array([max(x, y) for x, y in zip(scores, data[idx])])
        else:
            rscores = np.array([max(x, y) for x, y in zip(rscores, data[idx])])
    return scores, rscores

### 13-3. 단순 선호도 및 Softmax 함수 기반

In [1]:
def get_new_score_softmax(data, scores, idx_list, val_list, peak):
    new_scores = np.zeros(len(scores))
    for idx, val in zip(idx_list, val_list):
        if val > 0:
            new_scores += peak ** data[idx]
        else:
            new_scores += peak ** (1 - data[idx])
    return np.array([max(x, y) for x, y in zip(scores, new_scores / (peak * len(idx_list)))])

### 13-4. 교차 선호도 및 binary exponential 함수 기반

In [2]:
def get_new_score_crossexp(data, scores, rscores, idx_list, val_list, peak):
    cnt = 0
    for val in val_list:
        if not val > 0:
            cnt += 1
    new_scores = np.zeros(len(scores))
    new_rscores = np.zeros(len(scores))
    
    for idx, feature in enumerate(data[idx_list].T.copy()):
        feature = sorted(np.array([x if y > 0 else -x for x, y in zip(feature, val_list)]))
        for i in range(cnt):
            new_rscores[idx] += feature[i] * (peak ** (cnt - i - 1))
        for i in range(cnt, len(idx_list)):
            new_scores[idx] += feature[i] * (peak ** (i - cnt))

    new_rscores /= (peak ** cnt) - 1
    new_rscores *= peak - 1
    new_rscores *= -1
    new_scores /= (peak ** (len(idx_list) - cnt)) - 1
    new_scores *= peak - 1
    new_rscores = np.array([max(x, y) for x, y in zip(rscores, new_rscores)])
    new_scores = np.array([max(x, y) for x, y in zip(scores, new_scores)])
    
    return new_scores, new_rscores

## 14. 최종 선호도 결과에 기반한 추천

### 14-1. Default type

In [ ]:
def show_image_by_score_cross(file_names, scores, rscores, length = 25, reverse = False):
    if reverse:
        new_idx = sorted(range(len(scores)), key= lambda i: scores[i] - rscores[i])[:length]
    else:
        new_idx = sorted(range(len(scores)), key= lambda i: scores[i] - rscores[i])[-length:]
        new_idx.reverse()
    print(scores[new_idx] - rscores[new_idx])
    show_image_sampled(file_names, new_idx, plotsize = (5, 5), figsize = (16, 16))
    return new_idx

### 14-2. Remove Duplicate

In [ ]:
def show_image_by_score_cross_no_duplicated(file_names, scores, rscores, dup_list, length = 25, reverse = False):
    new_idx = sorted(range(len(scores)), key= lambda i: scores[i] - rscores[i])
    if not reverse:
        new_idx.reverse()
    result_idx = []
    idx = 0
    while len(result_idx) != length:
        if new_idx[idx] not in dup_list:
            result_idx.append(new_idx[idx])
        idx += 1
    print(scores[result_idx] - rscores[result_idx])
    show_image_sampled(file_names, result_idx, plotsize = (5, 5))
    return result_idx

### 14-3. Subfunction for check duplication

In [ ]:
def add_duplicated_list(idx_list, val_list, dup_list):
    for idx, val in zip(idx_list, val_list):
        if val > 0:
            dup_list.append(idx)
    return dup_list

## 12~14. Simulating Recommendation System

In [4]:
def recommend_images_v1(data_path, img_path):
    file_names = get_file_names(img_path, 'jpg')
    fmap = np.load(data_path)
    
    scores = np.zeros(len(fmap))
    rscores = np.zeros(len(fmap))
    dup_list = []
    
    for i in range(5):
        idx_list = get_random_idx_cross(scores, rscores, 30)
        show_image_sampled(file_names, idx_list)
        val_list = list(map(int, input().split()))
        dup_list = add_duplicated_list(idx_list, val_list, dup_list)
        scores, rscores = get_new_score_crossmax(new_fmap, scores, rscores, idx_list, val_list)
    
    result_idx = show_image_by_score_cross_no_duplicated(file_names, scores, rscores, dup_list)
    result_idx = show_image_by_score_cross_no_duplicated(file_names, scores, rscores, dup_list, reverse = True)